# Bring Your Own Data

In [41]:
import os
os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import evaluation.metrics as m
import evaluation.eval_framework as evu
import pandas as pd
import math

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

## Required fields
- `time_needed`: int
- `total_intervals`: int
- `power_rate`: float in kWh or MWh
- `session_start_time`: datetime64[ns] '2023-06-07 16:19:10'
- `end_start_time`: datetime64[ns] '2023-06-07 20:30:10'

### Notes

- Hints
   - evu.convert_to_utc() to convert a time expressed in any local time to UTC
   - evu.intervalize_power_rate() to convert a power rate from kilowatts to a 5-minute interval rate, optionally in megawatts
   - Use the sanitize functions to convert float to int

In [42]:
def sanitize_time_needed(x):
    return int(x / 300.0) * 5

def sanitize_total_intervals(x):
    return math.ceil(x)

In [43]:
example_df = pd.DataFrame([['2023-01-03 03:17:26',
        '2023-01-03 09:32:52.456704',
        326.4],
       ['2023-01-04 06:20:50',
        '2023-01-04 13:28:55.231872',
        326.4],
       ['2023-01-13 11:05:16',
        '2023-01-13 18:08:20.434752',
        326.4],
       ['2023-01-14 13:56:37',
        '2023-01-14 22:01:04.455693',
        326.4],
       ['2023-01-19 07:52:55',
        '2023-01-19 16:51:48.332139',
        326.4]],
        columns = ["session_start_time","session_end_time","power_rate"]
)

example_df["session_start_time"] = pd.to_datetime(example_df["session_start_time"])
example_df["session_end_time"] = pd.to_datetime(example_df["session_end_time"])

In [44]:
example_df

,session_start_time,session_end_time,power_rate
0,2023-01-03 03:17:26,2023-01-03 09:32:52.456704,326.4
1,2023-01-04 06:20:50,2023-01-04 13:28:55.231872,326.4
2,2023-01-13 11:05:16,2023-01-13 18:08:20.434752,326.4
3,2023-01-14 13:56:37,2023-01-14 22:01:04.455693,326.4
4,2023-01-19 07:52:55,2023-01-19 16:51:48.332139,326.4


In [45]:
example_df["length_of_session_in_seconds"] = (
        example_df.session_end_time - example_df.session_start_time
    ) / pd.Timedelta(seconds=1)

example_df["total_intervals"] = example_df["length_of_session_in_seconds"] / 300
example_df["total_intervals"] = example_df["total_intervals"].apply(lambda x: sanitize_total_intervals(x))
example_df["time_needed"] = example_df["length_of_session_in_seconds"].apply(lambda x: sanitize_time_needed(x))

example_df["power_rate"] = example_df["power_rate"].apply(lambda x: evu.intervalize_power_rate(x,convert_to_MW = True))

In [46]:
example_df

,session_start_time,session_end_time,power_rate,length_of_session_in_seconds,total_intervals,time_needed
0,2023-01-03 03:17:26,2023-01-03 09:32:52.456704,0.0272,22526.456704,76,375
1,2023-01-04 06:20:50,2023-01-04 13:28:55.231872,0.0272,25685.231872,86,425
2,2023-01-13 11:05:16,2023-01-13 18:08:20.434752,0.0272,25384.434752,85,420
3,2023-01-14 13:56:37,2023-01-14 22:01:04.455693,0.0272,29067.455693,97,480
4,2023-01-19 07:52:55,2023-01-19 16:51:48.332139,0.0272,32333.332139,108,535


In [ ]:
# Required fields are time_needed, total_intervals, power_rate, and session_start_time
FILE_PATH = ''
REGION = '' # i.e. PJM_DC

df = pd.read_csv(FILE_PATH)

In [ ]:
df["moer_forecast"] = df.apply(
    lambda x: evu.get_historical_fcst_data(
        sanitize_time_needed(x.session_start_time), 
        sanitize_total_intervals(x.total_intervals),
        region=REGION
    ),
    axis=1,
)

df["moer_actual"] = df.apply(
    lambda x: evu.get_historical_actual_data(
        sanitize_time_needed(x.session_start_time), 
        sanitize_total_intervals(x.total_intervals),
        region=REGION
    ),
    axis=1,
)

In [ ]:
def run_optimization_functions(
        df: pd.DataFrame, 
        contiguous=False,
        time_needed_col:str = "time_needed",
        total_intervals_col:str = "total_intervals",
        power_rate_col:str = "power_rate"
        ):
    # Lambda functions for common operations
    get_charging_schedule = lambda x: x["usage"].values.flatten()
    get_total_emissions = lambda x: x["emissions_co2e_lb"].sum()
    
    # Sanitize input data
    df["sanitize_intervals_plugged_in"] = df.apply(lambda x: sanitize_total_intervals(x.total_intervals_plugged_in), axis=1)
    df["sanitize_time_needed"] = df.apply(lambda x: sanitize_time_needed(x.total_seconds_to_95, x.length_of_session_in_seconds), axis=1)
    
    # Helper function for charge_per_interval
    def charge_per_interval_func(x, contiguous=False):
        return [(0, x)] if contiguous else []
    
    # Function to apply optimization
    def apply_optimization(row, moer_data, optimization_method):
        return evu.get_schedule_and_cost_api(
            usage_power_kw=row.power_output_rate,
            time_needed=row.sanitize_time_needed,
            total_time_horizon=row.sanitize_intervals_plugged_in,
            moer_data=moer_data,
            optimization_method=optimization_method,
            charge_per_interval=charge_per_interval_func(row.sanitize_time_needed, contiguous=contiguous)
        )
    
    # Apply optimizations
    df["optimizer_baseline"] = df.apply(lambda x: apply_optimization(x, x.moer_actual, "baseline"), axis=1)
    df["optimizer_ideal"] = df.apply(lambda x: apply_optimization(x, x.moer_actual, "auto"), axis=1)
    df["optimizer_simple"] = df.apply(lambda x: apply_optimization(x, x.moer_forecast, "auto"), axis=1)
    
    # Calculate emissions and charging schedules
    df["charging_schedule"] = df["optimizer_simple"].apply(get_charging_schedule)
    df["baseline_emissions"] = df["optimizer_baseline"].apply(get_total_emissions)
    df["ideal_charging_schedule"] = df["optimizer_ideal"].apply(get_charging_schedule)
    df["ideal_emissions"] = df["optimizer_ideal"].apply(get_total_emissions)
    df["forecast_emissions"] = df["optimizer_simple"].apply(get_total_emissions)
    
    # Calculate actual emissions
    df["actual_emissions"] = df.apply(
        lambda x: evu.get_total_emission(
            x.moer_actual["value"],
            x.optimizer_simple.energy_usage_mwh
        ),
        axis=1
    )
    
    return df